In [1]:
import os, sys, logging, importlib, math
import rasterio, affine

import geopandas as gpd
import pandas as pd

from zipfile import ZipFile
from affine import Affine
from rasterio import features
from rasterio.mask import mask
from rasterio.features import rasterize

import GEP
from GEP import gep_results

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
import GEP
#GEP.get_assumptions("0_0_0_0_0_0")
GEP.get_assumptions("1_1_1_1_1_0")

['Population growth: High',
 'Electricity demand target: Top-down high',
 'Intermediate investment: Capped connections in 2025',
 'Grid generation cost: High',
 'PV cost: High',
 'Rollout: Least-cost nationwide']

In [4]:
scenario = "1_1_1_1_1_0"
scenariosFolder=f'/media/gost/DATA1/GEP/Scenarios/Scenario_{scenario}'
gFields = ['id','geometry','FinalElecCode2025','FinalElecCode2030', 'Pop2030']

countries = []
for x in os.listdir(scenariosFolder):
    if os.path.isdir(os.path.join(scenariosFolder, x)):
        countries.append(x[:4])


# Join scenario results to cluster shapefile

In [7]:
country = 'cm-1'
xx_so = gep_results(country, 
                    scenariosFolder=f'/media/gost/DATA1/GEP/Scenarios/Scenario_{scenario}',
                    clustersFolder = '/media/gost/DATA1/GEP/Clusters',
                    scenario=scenario)
inD = xx_so.join_results()
inD.head()

,id,geometry,Pop,GridCellArea,Country,ElecPop,WindVel,GHI,TravelHours,Elevation,...,MinimumOverallCode2030,InvestmentCost2030,ElecStatusIn2030,InvestmentCapita2030,GridElecIn2030,OffGridElecIn2030,FinalElecCode2030,NewCapacity2030,GridPop2030,OffGridPop2030
0,1,"POLYGON ((10.27547285258622 5.378472222222223,...",1.610,0.010,Cameroon,0.000,3.240,1954.000,0.380,1638.000,...,3.000,928.789,1,472.595,0,1,3,0.285,0.000,1.965
1,2,"POLYGON ((10.27880630696881 5.359305555555555,...",1.610,0.010,Cameroon,0.000,2.690,1946.660,0.390,1655.040,...,3.000,932.291,1,474.377,0,1,3,0.286,0.000,1.965
2,3,"POLYGON ((10.26463912584281 5.379305555555556,...",4.410,0.010,Cameroon,0.000,3.030,1950.000,0.400,1608.000,...,3.000,2549.293,1,473.564,0,1,3,0.782,0.000,5.383
3,4,"POLYGON ((10.96716463697328 5.378472222222223,...",6.180,0.010,Cameroon,0.000,2.520,2035.000,1.250,748.000,...,3.000,3423.260,1,453.784,0,1,3,1.050,0.000,7.544
4,5,"POLYGON ((12.78889745705778 5.380972222222224,...",4.210,0.010,Cameroon,0.000,2.030,1922.000,4.560,665.700,...,3.000,2469.133,1,480.463,0,1,3,0.757,0.000,5.139


In [14]:
inD.to_file('/media/gost/DATA1/GEP/Clusters/GEP_clusters_%s.shp' % scenario)

In [11]:
tempD = inD.loc[inD['Pop'] > 10,['FinalElecCode2025','FinalElecCode2030','Population','geometry']]
tempD.shape

allRes = {}
cnt = 0
for country in countries:
    if not country in allRes.keys():
        print(f'Processing {country}')
        xx_so = gep_results(country, 
                    scenariosFolder=f'/media/gost/DATA1/GEP/Scenario_{scenario}',
                    clustersFolder = '/media/gost/DATA1/GEP/Clusters',
                    scenario=scenario)
        inD = xx_so.join_results()
        # Identify rows where the electrified source changes between 2025 and 2030
        changing_source = (inD['FinalElecCode2025'] > inD['FinalElecCode2030']) & (inD['FinalElecCode2025'] != 99)
        # For those changed rows, set the adjusted 2030 value = 2025 value
        inD['adjust_2030'] = inD['FinalElecCode2030']
        inD['ElecPop2030'] = inD['Pop2030'] - inD['ElecPop']
        inD.loc[changing_source,'adjust_2030'] = inD.loc[changing_source, 'FinalElecCode2025']
        allRes[country] = [inD.loc[:,['adjust_2030','ElecPop2030','ElecPop','Pop']].groupby('adjust_2030').sum(),
                           inD.loc[:,['FinalElecCode2030','ElecPop2030','ElecPop','Pop']].groupby('FinalElecCode2030').sum()]
    else:
        print(f'Already processed {country}')
    '''cnt += 1
    if cnt > 1:
        break'''

/home/gost/anaconda3/envs/Geog/lib/python3.7/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


(106696, 4)

In [3]:
allRes = {}
cnt = 0
for country in countries:
    if not country in allRes.keys():
        print(f'Processing {country}')
        xx_so = gep_results(country, 
                    scenariosFolder=f'/media/gost/DATA1/GEP/Scenario_{scenario}',
                    clustersFolder = '/media/gost/DATA1/GEP/Clusters',
                    scenario=scenario)
        inD = xx_so.join_results()
        # Identify rows where the electrified source changes between 2025 and 2030
        changing_source = (inD['FinalElecCode2025'] > inD['FinalElecCode2030']) & (inD['FinalElecCode2025'] != 99)
        # For those changed rows, set the adjusted 2030 value = 2025 value
        inD['adjust_2030'] = inD['FinalElecCode2030']
        inD['ElecPop2030'] = inD['Pop2030'] - inD['ElecPop']
        inD.loc[changing_source,'adjust_2030'] = inD.loc[changing_source, 'FinalElecCode2025']
        allRes[country] = [inD.loc[:,['adjust_2030','ElecPop2030','ElecPop','Pop','NewConnections2030','NewConnections2025']].groupby('adjust_2030').sum(),
                           inD.loc[:,['FinalElecCode2030','ElecPop2030','ElecPop','Pop','NewConnections2030','NewConnections2025']].groupby('FinalElecCode2030').sum()]
    else:
        print(f'Already processed {country}')
    '''cnt += 1
    if cnt > 1:
        break'''

Processing gm-1
Processing mw-1
Processing ao-1
Processing bd-1
Processing bf-1
Processing bi-1
Processing bj-1
Processing bw-1
Processing cd-1
Processing cf-1
Processing cg-1
Processing ci-1
Processing cm-1
Processing dj-1
Processing er-1
Processing et-1
Processing ga-1
Processing gh-1
Processing gn-1
Processing gq-1
Processing gw-1
Processing hn-1
Processing ht-1
Processing ke-1
Processing kh-1
Processing km-1
Processing lr-1
Processing ls-1
Processing mg-1
Processing ml-1
Processing mm-1
Processing mn-1
Processing mr-1
Processing mz-1
Processing na-1
Processing ne-1
Processing ng-1
Processing ni-1
Processing pg-1
Processing pk-1
Processing rw-1
Processing sd-1
Processing sl-1
Processing sn-1
Processing so-1
Processing ss-1
Processing st-1
Processing sz-1
Processing td-1
Processing tg-1
Processing tl-1
Processing tz-1
Processing ug-1
Processing za-1


/home/gost/anaconda3/envs/Geog/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


Processing zm-1
Processing zw-1


In [6]:
orig = allRes.copy()

In [12]:
final.head()

,ElecPop2030_adj_gm-1,ElecPop_adj_gm-1,Pop_adj_gm-1,NewConnections2030_adj_gm-1,NewConnections2025_adj_gm-1,ElecPop2030_orig_gm-1,ElecPop_orig_gm-1,Pop_orig_gm-1,NewConnections2030_orig_gm-1,NewConnections2025_orig_gm-1,...,ElecPop2030_adj_zw-1zw-1,ElecPop_adj_zw-1zw-1,Pop_adj_zw-1zw-1,NewConnections2030_adj_zw-1zw-1,NewConnections2025_adj_zw-1zw-1,ElecPop2030_orig_zw-1zw-1,ElecPop_orig_zw-1zw-1,Pop_orig_zw-1zw-1,NewConnections2030_orig_zw-1zw-1,NewConnections2025_orig_zw-1zw-1
adjust_2030,,,,,,,,,,,,,,,,,,,,,
1.000,1260560.309,891713.909,1298321.814,557459.001,382975.267,1549893.411,924960.277,1515524.876,595999.515,382975.267,...,7569157.216,5997957.648,8396983.382,4106178.933,2695735.855,11650593.508,6090095.403,11171968.650,4502333.682,2695735.855
3.000,33632.269,3077.965,28412.237,36710.234,0.000,33632.269,3077.965,28412.237,36710.234,0.000,...,3417961.854,54323.848,2308702.697,1660417.686,1811868.017,3417961.854,54323.848,2308702.697,1660417.686,1811868.017
5.000,851607.436,57826.139,634473.908,309317.831,317457.791,563630.164,24699.661,418412.995,270872.035,317457.791,...,5218048.497,99727.441,3535758.494,1565452.722,0.000,1164156.312,7773.599,779209.427,1171929.911,0.000
7.000,3302.061,169.867,2687.132,1468.184,622.740,1946.231,49.975,1544.983,1373.467,622.740,...,67169.904,415.561,44937.185,42489.381,0.000,39625.797,231.647,26500.984,39857.444,0.000


In [13]:
# del(final)

In [9]:
allRes = orig
for key, value in allRes.items():    
    combined = value[0].join(value[1], lsuffix="_adj", rsuffix='_orig')
    combined.columns = [f'{x}_{key}' for x in combined.columns]        
    try:
        final = final.join(combined, rsuffix=key)
    except:
        final = combined

In [18]:
os.getcwd()

'/home/gost/Github/gep_results_analysis/GEP_analysis'

In [19]:
final2.to_csv("final2.csv")

In [33]:
importlib.reload(GEP)
final2 = final.transpose()
final2['type'] = [x.split("_")[1] for x in final2.index]
final2['pop'] = [x.split("_")[0] for x in final2.index]
final2['continent'] = [GEP.get_continent(x.split("_")[2]) for x in final2.index]
final2.groupby(['type','pop','continent']).sum()

adjust_2030                                1.000        3.000         5.000  \
type pop                continent                                             
adj  ElecPop            AFR        363285446.185  1393182.873  14693449.338   
                        other      909872739.689  1804678.306  20813915.369   
     ElecPop2030        AFR        702415963.285 76879652.491 358332651.078   
                        other     1203757920.301 94869978.651 437612375.465   
     NewConnections2025 AFR        243785238.183 21348114.663  42573560.408   
                        other      410231761.171 24230582.753  51229101.938   
     NewConnections2030 AFR        348562893.940 56926344.325 100383034.206   
                        other      498625555.217 72445697.841 127671664.108   
     Pop                AFR        664006164.461 57081571.903 264637149.193   
                        other     1449094957.967 71036869.607 329765802.979   
orig ElecPop            AFR        371799052.961  1393308.320   6198591.602   
                        other      922604829.856  1804803.752   8109584.496   
     ElecPop2030        AFR        947610093.193 76881548.118 113969644.413   
                        other     1498672741.676 94871874.278 143787150.951   
     NewConnections2025 AFR        243785238.183 21348114.663  42573560.408   
                        other      410231761.171 24230582.753  51229101.938   
     NewConnections2030 AFR        371422503.989 56926344.325  77594184.236   
                        other      525715302.436 72445697.841 100667094.276   
     Pop                AFR        848121066.878 57083560.020  81145150.501   
                        other     1673810616.765 71038857.724 105894919.398   

adjust_2030                             7.000  
type pop                continent              
adj  ElecPop            AFR         81802.854  
                        other      182991.680  
     ElecPop2030        AFR       3419921.446  
                        other     5405454.552  
     NewConnections2025 AFR        372206.426  
                        other      756864.424  
     NewConnections2030 AFR       1336916.697  
                        other     2186243.073  
     Pop                AFR       2528039.725  
                        other     4136239.818  
orig ElecPop            AFR         67426.128  
                        other      161338.275  
     ElecPop2030        AFR       2763953.663  
                        other     4553025.814  
     NewConnections2025 AFR        372206.426  
                        other      756864.424  
     NewConnections2030 AFR       1281601.595  
                        other     2120403.017  
     Pop                AFR       2032358.943  
                        other     3469309.808

In [30]:
final2.loc['ElecPop2030_orig_bi-1']

adjust_2030
1.0         14618565.593
3.0           284447.795
5.0           178796.645
7.0                  NaN
type                orig
pop          ElecPop2030
continent            AFR
Name: ElecPop2030_orig_bi-1, dtype: object

In [25]:
final2.continent.unique()

array(['AFR', 'other'], dtype=object)

In [32]:
final2.to_csv("final2.csv", sep=";", na_rep=0)

In [22]:
final3 = final2.groupby(['type','pop','continent']).sum()

In [27]:
final3.to_csv("final3.csv", sep=";")

In [ ]:
#del(final)
final.to_csv(f'{scenariosFolder}_adjusted2.csv')

In [ ]:
adjusted_values = value[0]    
    adjusted_values.columns = [f'{key}_{x}' for x in adjusted_values.columns]
    orig_values = value[1]
    orig_values.columns = [f'{key}_{x}' for x in orig_values.columns]
    try:
        final_adj = final_adj.join(adjusted_values)
    

In [ ]:
value[0]

In [ ]:
inD.loc[inD['Pop2030'] > 100,].shape